In [2]:
import numpy as np
import warnings
from functools import wraps
from typing import Union

In [47]:
class SimpleTensor:
    def __init__(self, data, requires_grad=False, parents=None, creation_op=None):
        self.data = np.array(data) # hold the data
        self.requires_grad = requires_grad # flag to check if gradient is required
        self.parents = parents or [] # hold the parents
        self.grad = None # hold the gradient (this should only stay None if requires_grad is False)
        self.creation_op = creation_op # hold the operation that created the tensor

        if self.requires_grad: # if gradient is required, zero initialize the gradient
            self.grad = np.zeros_like(self.data)

    @property
    def backward_ops(self):
        """
        I did this to clearly see what's implemented and what's not
        """
        ops = {
            "add": self.backward_add,
            "mul": self.backward_mul
        }
        return ops
    
    def make_tensor(func):
        """
        Decorator to convert the 'other' arg to a tensor if its not already a tensor
        """
        @wraps(func) # does this line matte?
        def wrapper(self, other):
            if not isinstance(other, SimpleTensor):
                other = SimpleTensor(other)
            return func(self, other)
        return wrapper

    @make_tensor
    def __add__(self, other):
        data = np.add(self.data, other.data)
        return SimpleTensor(data, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="add")
    
    def backward_add(self):
        self.parents[0].backward(self.grad)
        self.parents[1].backward(self.grad)
    
    @make_tensor
    def __mul__(self, other):
        data = np.multiply(self.data, other.data)
        return SimpleTensor(data, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="mul")
    
    def backward_mul(self):
        grad_wrt_first_parent = self.grad * self.parents[1].data
        grad_wrt_second_parent = self.grad * self.parents[0].data
        self.parents[0].backward(grad_wrt_first_parent)
        self.parents[1].backward(grad_wrt_second_parent)

    def backward(self, grad=None):
        if not self.requires_grad: # if gradient is not required, return
            return
        
        if grad is None:  # if we call backward without passing a gradient, initialize the gradient to 1
            grad = np.ones_like(self.data)

        if self.grad is None:  # initialize self.grad if it's None
            self.grad = grad
        else:
            self.grad += grad  # accumulate gradient

        if self.creation_op is None: # if the tensor was created by the user
            return
        
        # run the appropriate backward operation
        backward_op = self.backward_ops.get(self.creation_op, None)
        if backward_op is not None:
            backward_op() # call it
        else:
            raise NotImplementedError("Only addition and multiplication implemented")
        
    def __repr__(self):
        return f"SimpleTensor(data={self.data}, requires_grad={self.requires_grad}, grad={self.grad})"

In [48]:
def test_vector_addition_backward():
    a = SimpleTensor([1, 2, 3], requires_grad=True)
    b = SimpleTensor([4, 5, 6], requires_grad=True)

    # Adding two tensors
    c = a + b

    # Calling backward on the result
    c.backward()

    assert np.all(a.grad == np.ones_like(a.data))
    assert np.all(b.grad == np.ones_like(b.data))

test_vector_addition_backward()

In [49]:
m1 = SimpleTensor([[1, 2], [3, 4]], requires_grad=True)
m2 = SimpleTensor([[5, 6], [7, 8]], requires_grad=True)
v = SimpleTensor([9, 10], requires_grad=True)

# multiplying two tensors
m3 = m1 * m2 * 2

m3.backward()

print(m1.grad)
print(m2.grad)

[[10 12]
 [14 16]]
[[2 4]
 [6 8]]


In [65]:
class Tensor:
    def __init__(self, data: Union[int, float, np.ndarray], dtype: str = 'float', requires_grad: bool = False, parents=None, creation_op=None):
        self.data = np.array(data, dtype=dtype) # The data contained in the tensor
        self.dtype = dtype # The data type of the tensor

        self.shape = self.data.shape # The shape of the tensor
        self.ndim = self.data.ndim # The number of dimensions of the tensor
        self.size = self.data.size # The number of elements in the tensor

        self.requires_grad = requires_grad # Whether or not to compute gradients for this tensor
        self.grad = None # The gradient of this tensor (this should be an array, like the data attribute)

        if self.requires_grad: # If we need to compute gradients for this tensor
            self.zero_grad() # Initialize the gradient to 0

        self.parents = parents or [] # Tensors from which this one was created
        self.creation_op = creation_op # The operation that created this tensor

    @property
    def backward_ops(self):
        """
        I did this to clearly see what's implemented and what's not
        """
        ops = {
            "add": self.backward_add,
            "sub": self.backward_sub,
            "mul": self.backward_mul,
            "div": self.backward_div,
            "pow": self.backward_pow,
            "matmul": self.backward_matmul,
            "transpose": self.backward_transpose,
        }
        return ops

    def zero_grad(self):
        """
        Zero out the gradient
        """
        self.grad = np.zeros_like(self.data)

    def backward(self, grad : np.ndarray = None):
        # the grad should be an array (not a Tensor) just like the data attribute
        """
        This function will be called recursively to backpropogate gradients (auto differentiation)
        """
        if not self.requires_grad: # if gradient is not required, return
            return
        
        # if we dont have a gradient passed in, initialize it to 1
        if grad is None:  # if we call backward without passing a gradient, initialize the gradient to 1
            grad = np.ones_like(self.data)

        # if we do have a gradient passed in, either initalize self.grad or accumulate it
        if self.grad is None:
            self.grad = grad
        else:
            self.grad += grad  # accumulate gradient

        # if the tensor was created by the user, return
        if self.creation_op is None:
            return
    
        # time to backpropogate
        backward_op = self.backward_ops.get(self.creation_op, None) # get the correct backward op
        if backward_op:
            backward_op() # call the backward op
        else:
            raise NotImplementedError(f"Backward op for {self.creation_op} not implemented")
        
    def make_tensor(func):
        """
        Decorator to convert the 'other' arg to a tensor if its not already a tensor
        """
        @wraps(func) # make sure the wrapper function has the same name, docstring, etc. as the original function
        def wrapper(self, other):
            if not isinstance(other, Tensor):
                other = Tensor(other)
            return func(self, other)
        return wrapper
            
    # Basic Operations ===========================================
    @make_tensor
    def __add__(self,  other: Union[int, float, 'Tensor']) -> 'Tensor':
        """
        a + b
        """
        result = np.add(self.data, other.data)
        return Tensor(result, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="add")
    
    def backward_add(self):
        """
        (a + b)' = a' + b'
        """
        self.parents[0].backward(self.grad) 
        self.parents[1].backward(self.grad)

    @make_tensor
    def __sub__(self, other: Union[int, float, 'Tensor']) -> 'Tensor':
        """
        a - b
        """
        result = np.subtract(self.data, other.data)
        return Tensor(result, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="sub")
    
    def backward_sub(self):
        """
        (a - b)' = a' - b'
        The first parent receives the gradient directly, the second parent receives the negation of the gradient.
        """
        self.parents[0].backward(self.grad)
        self.parents[1].backward(-self.grad)
    
    @make_tensor
    def __mul__(self, other: Union[int, float, 'Tensor']) -> 'Tensor':
        result = np.multiply(self.data, other.data)
        return Tensor(result, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="mul")
    
    def backward_mul(self):
        """
        (a * b)' = a' * b + a * b'
        The gradient is scaled by the other parent for each respective parent.
        """
        self.parents[0].backward(self.grad * self.parents[1].data) # a' * b
        self.parents[1].backward(self.grad * self.parents[0].data) # a * b'
    
    @make_tensor
    def __truediv__(self, other: Union[int, float, 'Tensor']) -> 'Tensor':
        result = np.divide(self.data, other.data)
        return Tensor(result, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="div")
    
    def backward_div(self):
        """
        (a / b)' = (a' * b - a * b') / b^2
        The first parent receives the scaled gradient, and the second parent receives the scaled and negated gradient.
        """
        self.parents[0].backward(self.grad / self.parents[1].data)  # a' / b
        self.parents[1].backward(-self.grad * self.parents[0].data / (self.parents[1].data ** 2))  # -a / b^2
        
    @make_tensor
    def __pow__(self, other: Union[int, float, 'Tensor']) -> 'Tensor':
        result = np.power(self.data, other.data)
        return Tensor(result, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="pow")
    
    def backward_pow(self):
        """
        f(a, b) = a^b
        df/da = b * a^(b - 1)
        df/db = a^b * ln(a)
        """
        a = self.parents[0].data
        b = self.parents[1].data

        # find partial derivatives
        grad_wrt_a = self.grad * b * (a ** (b - 1))
        grad_wrt_b = self.grad * (a ** b) * np.log(a)

        # backpropogate
        self.parents[0].backward(grad_wrt_a)
        self.parents[1].backward(grad_wrt_b)

    @make_tensor
    def __matmul__(self, other):
        """
        Matrix multiplication
        """
        result = np.matmul(self.data, other.data)
        return Tensor(result, requires_grad=(self.requires_grad or other.requires_grad), parents=[self, other], creation_op="matmul")

    def backward_matmul(self):
        """
        (A @ B)' = A' @ B + A @ B'
        """
        # find partial derivatives
        grad_wrt_first_parent = Tensor(np.matmul(self.grad, self.parents[1].data.T))
        grad_wrt_second_parent = Tensor(np.matmul(self.parents[0].data.T, self.grad))

        # backpropogate
        self.parents[0].backward(grad_wrt_first_parent.data)
        self.parents[1].backward(grad_wrt_second_parent.data)


    # Reverse Operations =========================================

    # Unary Operations ===========================================

    # no decorator because no args to convert to tensors
    def __neg__(self):
        return self * -1
    
    # no decorator because no args to convert to tensors
    def __abs__(self):
        return Tensor(np.abs(self.data), requires_grad=self.requires_grad, parents=[self], creation_op='abs')

    # Reduction Operations =======================================

    # Shape Operations ===========================================
    def transpose(self):
        return Tensor(self.data.transpose(), requires_grad=self.requires_grad, parents=[self], creation_op='transpose')

    def backward_transpose(self):
        """
        (A^T)' = (A')
        """
        self.parents[0].backward(self.grad.transpose()) # A'
    
    @property
    def T(self):
        return self.transpose()

    # Comparison Operations ======================================

    # Indexing, Slicing, Joining, Mutating Ops ==================

    # Utils =====================================================

    # Other =====================================================

    def __repr__(self):
        return f"Tensor({self.data}, requires_grad={self.requires_grad})"

In [66]:
m1 = Tensor([[1, 2, 3], [4, 5, 6]], requires_grad=True)
m2 = Tensor([[7, 8, 9], [10, 11, 12]], requires_grad=True)

m3 = m1 * m2

m3.backward()
print(m1.grad)
print(m2.grad)

[[ 7.  8.  9.]
 [10. 11. 12.]]
[[1. 2. 3.]
 [4. 5. 6.]]


In [76]:
m4 = Tensor([[1, 2, 3], [4, 5, 6]], requires_grad=True)
m5 = Tensor([[7, 8], [9, 10], [11, 12]], requires_grad=True)

m6 = m4 @ m2.T
m6.backward()

print(m4.grad)
print(m5.grad)

[[17. 19. 21.]
 [17. 19. 21.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]


In [35]:
m1 

Tensor([[22. 28.]
 [49. 64.]], requires_grad=True)

In [57]:
m3 = m1 @ m2
m4 = m3 * 2

In [58]:
m4.backward()

UFuncTypeError: Cannot cast ufunc 'add' output from dtype('O') to dtype('float64') with casting rule 'same_kind'